In [11]:
import numpy
import pandas as pd
import time
import re
import traceback
import psutil
import subprocess
import platform
import sys
import socket
import json
import os
import __main__
import __file__
import inspect
import gspread # this is for interacting with Google Sheets
from gspread_dataframe import get_as_dataframe, set_with_dataframe  # read and write to Google Sheets
# we need to authorize a user to interact with sheets
from googleapiclient.discovery import build
from google.oauth2 import service_account
from oauth2client.service_account import ServiceAccountCredentials
from google.colab import auth
from google.colab import userdata

# Get the secret value
secret = userdata.get('SERVICE_ACCOUNT_KEY')

# Load credentials from the JSON string
with open('/dev/stdin', 'w') as f:
    f.write(secret)
creds = service_account.Credentials.from_service_account_file('/dev/stdin')

ModuleNotFoundError: No module named '__file__'

In [14]:
##### initialize with kshsaa-sb-service credentials #####
print(os.getcwd())
def get_credentials():
    path = os.path.abspath(__file__)
    current_dir = os.path.dirname(os.path.abspath(__file__))

    # Construct the path to the JSON file relative to the current directory
    json_file_path = os.path.join(current_dir, 'config_variables.json')

    # Now you can use the relative path to the JSON file
    scope = ['https://www.googleapis.com/auth/spreadsheets']
    credentials = ServiceAccountCredentials.from_json_keyfile_name(
        json_file_path, scope)
    return credentials

# Call the function to get the credentials
credentials = get_credentials()
gc = gspread.authorize(credentials)

/
/


NameError: name '__file__' is not defined

In [3]:
### initialize a scoreboard in Pandas ###
# Example initialization of the scoreboard
pools = 2
teams_per_pool = 8
rounds = teams_per_pool - 1  # Total number of rounds
rooms_per_round = 8  # Number of rooms per round
columns = ['Room', 'Team 1', 'Score 1', 'Team 2', 'Score 2', 'Winner', 'Loser']
scoreboard = pd.DataFrame(columns=columns)

for r in range(1, rounds + 1):
    for room in range(1, rooms_per_round + 1):
        row = {'Room': f'Room {room}'}
        scoreboard = scoreboard.append(row, ignore_index=True)


def internet_connection_check(host="8.8.8.8", port=53, timeout=3):
    """
    Check internet connection by trying to connect to Google's DNS.
    """
    try:
        socket.create_connection((host, port), timeout=timeout)
        return True
    except Exception as ex:
        print(f"No internet connection: {ex}")
        return False

<ipython-input-3-d795401b5e6e>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scoreboard = scoreboard.append(row, ignore_index=True)
<ipython-input-3-d795401b5e6e>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scoreboard = scoreboard.append(row, ignore_index=True)
<ipython-input-3-d795401b5e6e>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scoreboard = scoreboard.append(row, ignore_index=True)
<ipython-input-3-d795401b5e6e>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scoreboard = scoreboard.append(row, ignore_index=True)
<ipython-input-3-d795401b5e6e>:13: FutureWarning: The frame.append method is deprecated and will be 

In [4]:
### initialize a dictionary to store all question values in a Pandas dataframe ###
Bowl = {f"Room {i}": {} for i in range(1, rooms_per_round + 1)}

###example of accessing the dataframe from room 6 round 4:
# Bowl["Room 6"]["Round 4"]

In [5]:
class Room:
    def __init__(self, url):
        self.url = url
        # Initialize the connection to the Google Sheet
        self.sheet = gc.open_by_key(url)

    def read_entire_sheet(self, worksheet):
        """Read the entire worksheet and return a pandas DataFrame."""
        rmdata = worksheet.get_all_values()
        return pd.DataFrame(rmdata)

    def batch_write_updates(self, worksheet, updates):
        """Batch write updates to the worksheet."""
        cell_list = []
        for cell_addr, value in updates.items():
            cell = worksheet.acell(cell_addr)
            cell.value = value
            cell_list.append(cell)
        worksheet.update_cells(cell_list)

    def read_current_scores(self, df):
        """Read the current scores and team names from the room."""

        # Assuming team names and scores are in the first row and 19th row respectively
        team1 = df.iloc[0, 1]  # B1
        score1 = df.iloc[18, 1]  # B19
        team2 = df.iloc[0, 3]  # D1
        score2 = df.iloc[18, 3]  # D19

        return {team1: int(score1), team2: int(score2)}

    def get_round_number(self, df):
        """get the round number for that room"""
        round = df.iloc[0,0]
        round_number = int(round.split()[1])
        return round_number

    def check_round_completion(self, df):
        """
        Check if the current round is completed.
        Returns:
            bool: True if the round is completed, False otherwise.
        """
        status = df.iloc[20,1]
        return status.lower() == 'end'

    def reset_room(self, round_number):
        """
        Prepare the reset for the next round and return the updates as a dictionary.
        """
        # Clearing the cells in B and D columns separately. and the end indicator
        cells_to_clear = ['B' + str(i) for i in range(2, 19)] + ['D' + str(i) for i in range(2, 19)] + ['B21']

        updates = {cell: '' for cell in cells_to_clear}

        # Increment round number
        updates['A1'] = f'Round {round_number}'

        return updates

    def update_teams(self, team_names):
        """
        Prepare the team names update for the new round and return the updates as a dictionary.
        Args:
            team_names (tuple): A tuple containing the names of the two teams.
        """
        updates = {
            'B1': team_names[0],
            'D1': team_names[1]
        }

        return updates

    def combine_room_updates(self, room, team_names, round_number):
        """
        Combine the updates from reset_room and update_teams into a single dictionary.
        Args:
            room: The Room object.
            team_names (tuple): The names of the teams for the new round.
        Returns:
            dict: A dictionary containing all the cell updates.
        """
        reset_updates = room.reset_room(round_number)
        team_updates = room.update_teams(team_names)
        return {**reset_updates, **team_updates}

    def store_room_data(self, room_number, round_number, df):
        room_key = f"Room {room_number}"
        round_key = f"Round {round_number}"
        Bowl[room_key][round_key] = df

In [9]:
class ScoreboardManager:
    def __init__(self, scoreboard_url):
        self.scoreboard_url = scoreboard_url
        self.scoreboard_sheet = gc.open_by_key(scoreboard_url, scope='https://www.googleapis.com/auth/spreadsheets')

    def read_entire_scoreboard(self):
        """Read the entire scoreboard and return a pandas DataFrame."""
        worksheet = self.scoreboard_sheet.get_worksheet(0)
        sbdata = worksheet.get_all_values()
        return pd.DataFrame(sbdata)

    def batch_write_scoreboard_updates(self, updates):
        """Batch write updates to the scoreboard."""
        worksheet = self.scoreboard_sheet.get_worksheet(0)
        cell_list = []
        for cell_addr, value in updates.items():
            cell = worksheet.acell(cell_addr)
            cell.value = value
            cell_list.append(cell)
        worksheet.update_cells(cell_list)


    def get_teams_from_pools(self, df):
        pool_a_teams = df.iloc[2:2 + teams_per_pool, 0].tolist()  # Teams in column A (Pool A)
        pool_b_teams = df.iloc[2:2 + teams_per_pool, 5].tolist()  # Teams in column F (Pool B)

        pools = {
            'Pool A': pool_a_teams,
            'Pool B': pool_b_teams
        }
        return pools


    def calculate_win_loss_tallies_per_pool(self, df):
        pool_teams = self.get_teams_from_pools(df)

        # Initialize win_loss_tallies
        win_loss_tallies = {pool: {team: {'Wins': 0, 'Losses': 0} for team in teams} for pool, teams in pool_teams.items()}

        # Define the start and end row for the range
        start_row = teams_per_pool + 3  # For F12 (index 11 since pandas uses 0-based indexing)
        end_row = start_row + (rooms_per_round * rounds) + (2 * (rounds - 1)) # For G79

        # Iterate through the specified range in the DataFrame to count wins and losses
        for index in range(start_row, end_row + 1):
            winner = df.iloc[index, 5]  # Column F (winner) is at index 5
            loser = df.iloc[index, 6]  # Column G (loser) is at index 6

            # Update tally for winner
            for pool, teams in pool_teams.items():
                if winner in teams:
                    win_loss_tallies[pool][winner]['Wins'] += 1

            # Update tally for loser
            for pool, teams in pool_teams.items():
                if loser in teams:
                    win_loss_tallies[pool][loser]['Losses'] += 1

        return win_loss_tallies




    def calculate_proto_rankings(self, win_loss_tallies):
        proto_rankings = {}
        for pool, teams_tallies in win_loss_tallies.items():
            # Group teams based on their win-loss record
            record_groups = {}
            for team, tallies in teams_tallies.items():
                record = (tallies['Wins'], tallies['Losses'])
                if record not in record_groups:
                    record_groups[record] = []
                record_groups[record].append(team)

            # Sort the record groups
            sorted_records = sorted(record_groups.keys(), key=lambda x: (-x[0], x[1]))

            # Assign rankings within each record group
            pool_rankings = {}
            current_rank = 1
            for record in sorted_records:
                for team in record_groups[record]:
                    pool_rankings[team] = {'Rank': current_rank, 'Wins': record[0], 'Losses': record[1]}
                current_rank += len(record_groups[record])  # Increment rank for the next group

            proto_rankings[pool] = pool_rankings

        return proto_rankings


    def prepare_pool_updates(self, win_loss_tallies, proto_rankings, pool_start_col):
        updates = {}
        row_offset = 3  # Assuming team data starts from the 3rd row

        for team, tallies in win_loss_tallies.items():
            rank = proto_rankings[team]['Rank']
            wins = tallies['Wins']
            losses = tallies['Losses']

            # Calculate row based on team order
            team_row = row_offset + list(win_loss_tallies.keys()).index(team)

            # Prepare cell locations based on pool_start_col
            wins_cell = f'{pool_start_col}{team_row}'
            losses_cell = f'{chr(ord(pool_start_col) + 1)}{team_row}'
            rank_cell = f'{chr(ord(pool_start_col) + 2)}{team_row}'

            # Add updates
            updates[wins_cell] = wins
            updates[losses_cell] = losses
            updates[rank_cell] = rank

        return updates


    '''
    def refresh_scoreboard(self, rooms, df, roomdf):
        # Read the entire scoreboard into a DataFrame
        updates = {}
        print("i somehow happened! im a ghost")
        for room_name, room in rooms.items():
            round_number = room.get_round_number(roomdf)
            room_data = room.read_current_scores(roomdf)
            updates.update(self.update_round_scoreboard(df, round_number, {room_name: room_data}))

            if room.check_round_completion(roomdf):
                round_number += 1
                next_round_teams = self.get_teams_for_room(df, round_number, room_name)
                updates.update(room.combine_room_updates(room, next_round_teams))'''


    def update_round_scoreboard(self, df, round_number, room_scores):
        updates = {}
        for room, scores in room_scores.items():
            room_row = self.find_room_row(df, room, round_number)
            team1, team2 = list(scores.keys())
            score1, score2 = scores[team1], scores[team2]
            print(room)
            print(scores, round_number)

            # Prepare updates in 'A1' notation
            updates[f'C{room_row}'] = score1  # Score for Team 1
            updates[f'E{room_row}'] = score2  # Score for Team 2

            winner = team1 if score1 > score2 else team2
            loser = team2 if score1 > score2 else team1

            updates[f'F{room_row}'] = winner
            updates[f'G{room_row}'] = loser

        return updates


    def find_room_row(self, df, room_name, round_number):
        start_row = self.determine_start_row_for_round(round_number)
        for i in range(start_row, start_row + 9):  # Assuming 8 rooms per round
            if df.iloc[i, 0] == room_name:  # Room name in column 1
                return i + 1  # Offset by 1 due to 0-based indexing


    def get_teams_for_room(self, df, round_number, room_name):
        room_row = self.find_room_row(df, room_name, round_number)
        if room_row is not None:
            # Assuming team names are in columns 2 and 4 (B and D)
            team1 = df.iloc[room_row - 1, 1]  # -1 due to 0-based indexing
            team2 = df.iloc[room_row - 1, 3]
            return team1, team2
        return None, None


    def determine_start_row_for_round(self, round_num):
        # Implement the logic to determine the start row for each round
        return (int(round_num) * (int(rooms_per_round) + 2)) + 1

In [10]:
# Initialize rooms with their URLs
print("Initializing rooms and scoreboard manager...")
time.sleep(2.5)
rooms = {
    "Room 1": Room('11rMSZaa0QhvJXZmPf8jytybQf3V20Qf32Opu6LGaNN4'),
    "Room 2": Room('1FTm_618MJ-joBktyiRDAWaA42pk7KsopycjN9zga7GU'),
    "Room 3": Room('1baxhrsJZbkauPRKwD0zEKmZyUlUFGXWE805mgu9bezM'),
    "Room 4": Room('1Z9b8FI5NXeLQPVGQ2yk1fqpPfuxE8mliiEsSJSgnaXg'),
    "Room 5": Room('1422wWNLvqm6wwJvmRstFigFMo4Qhi1L24Apy71fsKPM'),
    "Room 6": Room('1qobKfXSn2NNUZOV3Bmk-8CRJugF_wNN-XUYI4Jh1Okw'),
    "Room 7": Room('1g_4vkLjD4DDwiq44y1I4Y5ZhUGB0n2tGVzt8p29JYjI'),
    "Room 8": Room('12IRb4KXiH2O8xoj2ojJHac9xwpCachUmGJlbozDHVMA')
    # ... more rooms as needed
}

# Initialize ScoreboardManager with the URL of your Google Sheet
scoreboard_manager = ScoreboardManager('15002HkgDOaEhM9Ah5vTlhmQlFQfh8PVzBY_pySF4T10')

print("Rooms and scoreboard manager initialized")

time.sleep(10)

# Initialize room setup
for room_name, room in rooms.items():
  setup_successful = False
  while not setup_successful:
      try:
          # Read the entire room sheet into a DataFrame
          print(f"setting up {room_name}")
          worksheet = room.sheet.get_worksheet(0)
          roomdf = room.read_entire_sheet(worksheet)
          round_number = 5

          # Reset the room and prepare updates for the next round
          room_updates = room.reset_room(round_number)  # Reset the room for the next round

          # Set round number in A1
          room_updates['A1'] = 'Round ' + str(round_number)

          # Get teams for the first round and prepare team updates
          df = scoreboard_manager.read_entire_scoreboard()  # Read the entire scoreboard
          next_round_teams = scoreboard_manager.get_teams_for_room(df, round_number, room_name)
          team_updates = room.update_teams(next_round_teams)

          # Combine room updates and team updates
          combined_updates = {**room_updates, **team_updates}

          # Batch write updates to the room
          room.batch_write_updates(worksheet, combined_updates)

          print(f"{room_name} successfully set up for Round {round_number}.")
          setup_successful = True

      except gspread.exceptions.APIError as e:
          if e.response.status_code == 429:
              error_message = str(e)

              # Extract quota limit name and value using regular expressions
              quota_limit_name_search = re.search(r"'quota_limit': '([^']*)'", error_message)
              quota_limit_value_search = re.search(r"'quota_limit_value': '(\d+)'", error_message)

              if quota_limit_name_search and quota_limit_value_search:
                  quota_limit_name = quota_limit_name_search.group(1)
                  quota_limit_value = quota_limit_value_search.group(1)
                  print(f"API Quota Limit Exceeded. Limit Name: {quota_limit_name}, Limit Value: {quota_limit_value} requests per minute.")
              else:
                  print("API quota limit exceeded. Specific details not available.")

              print("Waiting for 30 seconds before retrying...")
              time.sleep(30)
              continue
          else:
              raise

      except Exception as e:
          print(f"An unexpected error occurred while setting up {room_name}.")
          print(f"Error type: {type(e).__name__}")
          print(f"Error message: {e}")
          print("Detailed traceback:")
          traceback.print_exc()


print("All rooms have been successfully set up and are ready for pool play.")

input("press ENTER to start pool play...")

Initializing rooms and scoreboard manager...


TypeError: Client.open_by_key() got an unexpected keyword argument 'scope'

In [8]:
# preliminary pool play loop
print("Starting pool play. Monitoring rounds...")
i = 1
while True:
    poolfinished = True
    # Initialize updates dictionary
    updates = {}
    try:
        # Read the entire scoreboard into a DataFrame
        df = scoreboard_manager.read_entire_scoreboard()

        break

        # Iterate through each room
        for room_name, room in rooms.items():
            # Read the entire room sheet into a DataFrame
            worksheet = room.sheet.get_worksheet(0)
            roomdf = room.read_entire_sheet(worksheet)

            # Get the current round number
            round_number = room.get_round_number(roomdf)

            # Check if the round is completed in the room
            if room.check_round_completion(roomdf):
                # Process end of round logic
                room.store_room_data(room_name.split()[1], round_number, roomdf)
                if round_number < rounds:
                  # Add scores to current scores and updates
                  scores = room.read_current_scores(roomdf)  # Get current scores
                  score_updates = scoreboard_manager.update_round_scoreboard(df, round_number, {room_name: scores})
                  updates.update(score_updates)
                  print(f"Round {round_number} in {room_name} is complete. Preparing for the next round.")
                  round_number += 1
                  next_round_teams = scoreboard_manager.get_teams_for_room(df, round_number, room_name)  # Get next round teams
                  room_updates = room.combine_room_updates(room, next_round_teams, round_number)  # Update teams for next round
                  room.batch_write_updates(worksheet, room_updates)

                elif round_number == rounds:
                  continue
            else:
                # Add scores to current scores and updates
                scores = room.read_current_scores(roomdf)  # Get current scores
                score_updates = scoreboard_manager.update_round_scoreboard(df, round_number, {room_name: scores})
                updates.update(score_updates)

            poolfinished = False

        # Print updates (for debugging)
        #print(updates)

        if poolfinished:
          win_loss_tallies = scoreboard_manager.calculate_win_loss_tallies_per_pool(df)
          proto_rankings = scoreboard_manager.calculate_proto_rankings(win_loss_tallies)
          pool_updates_A = scoreboard_manager.prepare_pool_updates(win_loss_tallies['Pool A'], proto_rankings['Pool A'], 'B')
          pool_updates_B = scoreboard_manager.prepare_pool_updates(win_loss_tallies['Pool B'], proto_rankings['Pool B'], 'G')
          updates.update(pool_updates_A)
          updates.update(pool_updates_B)
          scoreboard_manager.batch_write_scoreboard_updates(updates)
          print("all rounds are finished, moving onto the finals...")
          break

    except gspread.exceptions.APIError as e:
            # Handle specific API limit error
            if e.response.status_code == 429:
                print("API quota limit exceeded. Waiting for 30 seconds before retrying...")
                time.sleep(30)
                continue
            else:
                raise

    try:
        not_updated = True
        while not_updated:
            # Update the scoreboard with the collected updates
            win_loss_tallies = scoreboard_manager.calculate_win_loss_tallies_per_pool(df)
            proto_rankings = scoreboard_manager.calculate_proto_rankings(win_loss_tallies)
            pool_updates_A = scoreboard_manager.prepare_pool_updates(win_loss_tallies['Pool A'], proto_rankings['Pool A'], 'B')
            pool_updates_B = scoreboard_manager.prepare_pool_updates(win_loss_tallies['Pool B'], proto_rankings['Pool B'], 'G')
            updates.update(pool_updates_A)
            updates.update(pool_updates_B)
            scoreboard_manager.batch_write_scoreboard_updates(updates)
            not_updated = False

    except gspread.exceptions.APIError as e:
          # Handle specific API limit error
          if e.response.status_code == 429:
              print("API quota limit exceeded. Waiting for 30 seconds before retrying...")
              time.sleep(30)
              continue
          else:
              raise

    ## write here to access a pandas dataframe eg. Bowl["Room 6"]["Round 4"]


    # Wait before next iteration
    print(f"cycle {i} complete, please wait 30 seconds before cycle {i+1} begins")
    i += 1
    time.sleep(30)

Starting pool play. Monitoring rounds...


RefreshError: ('invalid_scope: Invalid OAuth scope or ID token audience provided.', {'error': 'invalid_scope', 'error_description': 'Invalid OAuth scope or ID token audience provided.'})

In [ ]:
class TieBreaker:
    def __init__(self, scoreboard_manager, rooms):
        self.scoreboard_manager = scoreboard_manager
        self.rooms = rooms
        self.final_rankings = {'A': {}, 'B': {}}  # Initialize for both pools


    def build_tournament_dataframe(self, df):
        rounds = (len(df) - (3 + teams_per_pool)) // (rooms_per_round + 2)
        columns = ['Team Name', 'Pool', 'Wins', 'Losses', 'Rank']
        for i in range(1, rounds + 1):
            columns += [f'Round_{i}_Opponent', f'Round_{i}_Score', f'Round_{i}_Result']

        tournament_df = pd.DataFrame(columns=columns)
        new_rows = []  # List to hold new rows

        # Populate initial team data for Pool A and Pool B
        for pool_index, pool_name in zip([0, 5], ['A', 'B']):
            for team_row in range(2, 2 + teams_per_pool):
                team_name = df.iat[team_row, pool_index]
                wins = df.iat[team_row, pool_index + 1]
                losses = df.iat[team_row, pool_index + 2]
                rank = df.iat[team_row, pool_index + 3]
                team_data = [team_name, pool_name, wins, losses, rank] + [None] * (3 * rounds)
                new_rows.append(pd.Series(team_data, index=columns))

        # Add the new rows to the DataFrame
        tournament_df = pd.concat([tournament_df, pd.DataFrame(new_rows)], ignore_index=True)

        # Populate round results
        for round_num in range(1, rounds + 1):
            round_start_row = 3 + teams_per_pool + (round_num - 1) * (rooms_per_round + 2)
            for room_row in range(round_start_row, round_start_row + rooms_per_round):
                room_info = df.iloc[room_row]
                team1, score1 = room_info[1], room_info[2]
                team2, score2 = room_info[3], room_info[4]
                winner = room_info[5]

                score_format_team1 = f"{score1}--{score2}"
                score_format_team2 = f"{score2}--{score1}"

                # Replace with the actual round number
                tournament_df.loc[(tournament_df['Team Name'] == team1), [f'Round_{round_num}_Opponent', f'Round_{round_num}_Score', f'Round_{round_num}_Result']] = [team2, score_format_team1, 'win' if winner == team1 else 'loss']
                tournament_df.loc[(tournament_df['Team Name'] == team2), [f'Round_{round_num}_Opponent', f'Round_{round_num}_Score', f'Round_{round_num}_Result']] = [team1, score_format_team2, 'win' if winner == team2 else 'loss']

        # Populate round results (adjust indices according to your DataFrame)
        for round_num in range(1, rounds + 1):
            round_start_row = 3 + teams_per_pool + (round_num - 1) * (rooms_per_round + 2)
            for room_row in range(round_start_row, round_start_row + rooms_per_round):
                room_info = df.iloc[room_row]
                team1, score1 = room_info[1], room_info[2]
                team2, score2 = room_info[3], room_info[4]
                winner = room_info[5]

                score_format_team1 = f"{score1}-{score2}"
                score_format_team2 = f"{score2}-{score1}"

                # Update the tournament DataFrame
                for team, score_format in [(team1, score_format_team1), (team2, score_format_team2)]:
                    tournament_df.loc[(tournament_df['Team Name'] == team),
                                      [f'Round_{round_num}_Opponent', f'Round_{round_num}_Score',
                                       f'Round_{round_num}_Result']] = [team2 if team == team1 else team1, score_format, 'win' if winner == team else 'loss']

        return tournament_df


    def calculate_final_rankings(self, df):
        proto_rankings = self.calculate_proto_rankings(df)

        # Initialize final rankings with all teams
        self.final_rankings = {'A': {}, 'B': {}}
        for pool, teams in proto_rankings.items():
            for team, data in teams.items():
                self.final_rankings[pool][team] = data['Rank']

        def resolve_pool_ties(pool):
            teams_with_ties = self.identify_ties(pool, self.final_rankings)
            for rank, tied_teams in teams_with_ties.items():
                self.resolve_ties_for_rank(tied_teams, pool, df, rank)

        # Resolve ties for each pool
        for pool in self.final_rankings:
            resolve_pool_ties(pool)

        # Re-check for remaining ties
        for pool in self.final_rankings:
            if self.identify_ties(pool, self.final_rankings):
                resolve_pool_ties(pool)
                break


    def extract_win_loss_tallies(self, df):
        win_loss_tallies = {'A': {}, 'B': {}}

        pool_start_col = {'A': 0, 'B': 5}  # Adjust based on your structure

        for pool in ['A', 'B']:
            for team_row in range(2, 2 + teams_per_pool):  # Starting from row 3 for each pool
                team_name = df.iat[team_row, pool_start_col[pool]]
                wins = df.iat[team_row, pool_start_col[pool] + 1]  # Wins column
                losses = df.iat[team_row, pool_start_col[pool] + 2]  # Losses column

                # Ensure wins and losses are strings that can be converted to integers
                if isinstance(wins, str) and wins.isdigit():
                    wins = int(wins)
                if isinstance(losses, str) and losses.isdigit():
                    losses = int(losses)

                win_loss_tallies[pool][team_name] = {'Wins': wins, 'Losses': losses}

        return win_loss_tallies


    def calculate_proto_rankings(self, df):
        proto_rankings = {'A': {}, 'B': {}}

        for pool in ['A', 'B']:
            pool_df = df[df['Pool'] == pool]

            for index, row in pool_df.iterrows():
                team = row['Team Name']
                rank = row['Rank']

                if isinstance(rank, str) and rank.isdigit():
                    rank = int(rank)

                proto_rankings[pool][team] = {'Rank': rank}

        return proto_rankings


    def identify_ties(self, pool, final_rankings):
        ties = {}
        for team, rank in final_rankings[pool].items():
            if rank not in ties:
                ties[rank] = [team]
            else:
                ties[rank].append(team)
        return {rank: teams for rank, teams in ties.items() if len(teams) > 1}


    def resolve_ties_for_rank(self, tied_teams, pool, df, start_rank):
        print(f"Resolving tie for rank {start_rank} in pool {pool}: {tied_teams}")
        resolved, new_rankings = self.head_to_head_comparison(df, tied_teams, pool)

        if resolved:
            print("head to head resolved tie")
            self.update_rankings(new_rankings, start_rank, pool)
        else:
            print("head to head failed to resolve tie")

            resolved, rankings = self.margin_of_victory_comparison(df, tied_teams, pool)
            if resolved:
                print("margin of victories resolved tie")
                self.update_rankings(rankings, start_rank, pool)
            else:
                print("margin of victory failed to resolve tie")

                resolved, rankings = self.total_points_comparison(df, tied_teams, pool)
                if resolved:
                    print("cumulative score resolved tie")
                    self.update_rankings(rankings, start_rank, pool)
                else:
                    print("cumulative scores failed to resolve tie")
                    self.call_for_additional_matches(tied_teams)


    def update_rankings(self, new_rankings, start_rank, pool):
        print(f"Updating rankings in pool {pool} starting from rank {start_rank}")

        # Sort the rankings based on scores in descending order
        sorted_rankings = sorted(new_rankings.items(), key=lambda x: x[1])

        current_rank = start_rank
        last_score = None
        rank_increment = 0

        for team, score in sorted_rankings:
            if score != last_score:
                last_score = score
                current_rank += rank_increment
                rank_increment = 1
            else:
                rank_increment += 1

            self.final_rankings[pool][team] = current_rank
            print(f"Updating team {team} to rank {current_rank}")

        print("\n")


    def head_to_head_comparison(self, df, tied_teams, pool):
        print(f"Performing head-to-head comparison for tie resolution in Pool {pool}: {tied_teams}")
        head_to_head_results = {team: {'wins': 0, 'losses': 0} for team in tied_teams}

        for team in tied_teams:
            for opponent in tied_teams:
                if team != opponent:
                    match_result = self.get_match_result(df, team, opponent, pool)
                    if match_result == 'win':
                        head_to_head_results[team]['wins'] += 1
                    elif match_result == 'loss':
                        head_to_head_results[team]['losses'] += 1
                    print(f"{team} vs {opponent}: {match_result}")
        print(head_to_head_results)

        # Sort teams based on head-to-head wins and then losses
        sorted_head_to_head = sorted(head_to_head_results.items(), key=lambda x: (-x[1]['wins'], x[1]['losses']))

        # Check if all tied teams have the same head-to-head record
        tied_teams_records = [(results['wins'], results['losses']) for results in head_to_head_results.values()]
        all_same_record = len(set(tied_teams_records)) == 1

        # If all teams have the same record, the tie is not resolved
        if all_same_record:
            print("All teams have the same head-to-head record. Tie not resolved.")
            return False, {}

        # Assign rankings, allowing for ties
        rankings = {}
        current_rank = 1
        for i, (team, result) in enumerate(sorted_head_to_head):
            if i == 0 or result != sorted_head_to_head[i-1][1]:
                current_rank = i + 1
            rankings[team] = current_rank

        # Check if head-to-head comparison resolves the tie
        resolved = self.is_resolved(rankings)
        return resolved, rankings


    def margin_of_victory_comparison(self, df, tied_teams, pool):
        print(f"Calculating margins of victory for tie resolution in Pool {pool}: {tied_teams}")
        margins = {team: 0 for team in tied_teams}
        for team in tied_teams:
            for opponent in tied_teams:
                if team != opponent:
                    margin, team_score, opponent_score = self.calculate_margin_of_victory(df, team, opponent, pool)
                    margins[team] += margin
                    print(f"Match {team} vs {opponent}: Team Score - {team_score}, Opponent Score - {opponent_score}, Margin - {margin}")

        # Check if all tied teams have the same margin of victory
        tied_teams_margins = [margins[team] for team in tied_teams]
        all_same_margin = len(set(tied_teams_margins)) == 1

        # If all teams have the same margin, the tie is not resolved
        if all_same_margin:
            print("All teams have the same margin of victory. Tie not resolved.")
            return False, {}

        # Sort teams based on their margins and assign ranks
        sorted_margins = sorted(margins.items(), key=lambda x: x[1], reverse=True)
        # Assign rankings, allowing for ties
        rankings = {}
        current_rank = 1
        for i, (team, margin) in enumerate(sorted_margins):
            if i == 0 or margin != sorted_margins[i-1][1]:
                current_rank = i + 1
            rankings[team] = current_rank

        print("Margins:", margins)
        return self.is_resolved(rankings), rankings


    def total_points_comparison(self, df, tied_teams, pool):
        print(f"Calculating total points for tie resolution in Pool {pool}: {tied_teams}")
        total_points = {team: self.calculate_total_points(df, team, pool) for team in tied_teams}

        # Check if all tied teams have the same total points
        tied_teams_points = [total_points[team] for team in tied_teams]
        all_same_total_points = len(set(tied_teams_points)) == 1

        if all_same_total_points:
            print("All teams have the same total points. Tie not resolved.")
            return False, {}

        # Calculate total points and sort teams
        sorted_points = sorted(total_points.items(), key=lambda x: x[1], reverse=True)

        # Assign rankings, allowing for ties
        rankings = {}
        current_rank = 1
        for i, (team, points) in enumerate(sorted_points):
            if i == 0 or points != sorted_points[i-1][1]:
                current_rank = i + 1
            rankings[team] = current_rank

        print("Total Points:", total_points)
        return self.is_resolved(rankings), rankings


    def call_for_additional_matches(self, tied_teams):
        print("\n")
        print(f"New match(es) required among the following teams: {', '.join(tied_teams)}")
        print("\n")


    def is_resolved(self, rankings):
        print("Rankings being checked for resolution:", rankings)
        # Extract only the ranks from the rankings dictionary
        ranks = list(rankings.values())
        print("Extracted ranks:", ranks)

        # Check if all ranks are not the same (i.e., at least one rank is different)
        all_ranks_not_same = len(set(ranks)) > 1
        print("Is there at least one different rank?", all_ranks_not_same)

        return all_ranks_not_same


    def get_match_result(self, df, team, opponent, pool):
        # Iterate over the DataFrame to find the specific match
        for i in range(1, len(df.columns) // 3):  # Assuming 3 columns per round
            team_row = df[(df['Team Name'] == team) & (df['Pool'] == pool)]
            opponent_row = df[(df['Team Name'] == opponent) & (df['Pool'] == pool)]

            if not team_row.empty and not opponent_row.empty:
                if team_row.iloc[0][f'Round_{i}_Opponent'] == opponent:
                    return 'win' if team_row.iloc[0][f'Round_{i}_Result'] == 'win' else 'loss'

        return 'tie'  # Return 'tie' if no match found or if data is missing


    def calculate_margin_of_victory(self, df, team, opponent, pool):
        margin = 0
        team_score, opponent_score = 0, 0
        for i in range(1, (len(df.columns) // 3) - 1):  # Assuming 3 columns per round
            team_row = df[(df['Team Name'] == team) & (df['Pool'] == pool)]
            if not team_row.empty:
                if team_row.iloc[0][f'Round_{i}_Opponent'] == opponent:
                    score = team_row.iloc[0][f'Round_{i}_Score']
                    if '-' in score:
                        score1, score2 = map(int, score.split('--'))
                        if team_row.iloc[0][f'Round_{i}_Result'] == 'win':
                            margin += abs(score1 - score2)
                            team_score, opponent_score = score1, score2
                        elif team_row.iloc[0][f'Round_{i}_Result'] == 'loss':
                            margin -= abs(score1 - score2)
                            team_score, opponent_score = score1, score2
        return margin, team_score, opponent_score


    def calculate_total_points(self, df, team, pool):
        print(f"Calculating total points for Team {team} in Pool {pool}")
        total_points = 0
        for i in range(1, (len(df.columns) // 3) - 1):  # Assuming 3 columns per round
            team_row = df[(df['Team Name'] == team) & (df['Pool'] == pool)]
            if not team_row.empty:
                score = team_row.iloc[0][f'Round_{i}_Score']
                if score and '-' in score:
                    team_score = int(score.split('--')[0])  # Extracting team's score
                    total_points += team_score
                    print(f"Round {i}: Score - {score}, Team Score - {team_score}")
                else:
                    print(f"Invalid or missing score for team {team} in round {i}")
        print(f"Total points for Team {team}: {total_points}")
        return total_points


    def print_final_rankings(self):
        print("Final Rankings:")
        for pool, teams_rankings in self.final_rankings.items():
            print(f"Final Rankings for {pool}:")
            # Sort the teams based on their rank (integer value)
            for team, rank in sorted(teams_rankings.items(), key=lambda x: x[1]):
                print(f"{team}: Rank {rank}")
            print("\n")


    def update_scoreboard_with_final_rankings(self, df):
        updates = {}
        pool_a_start_row = 1  # Starting row index for Pool A teams
        pool_b_start_row = 1  # Starting row index for Pool B teams
        pool_a_rank_col = 'D'  # Excel column letter for ranks in Pool A
        pool_b_rank_col = 'I'  # Excel column letter for ranks in Pool B

        for pool, teams_rankings in self.final_rankings.items():
            for team, rank in teams_rankings.items():
                if pool == 'A':
                    team_row_index = df.index[df[0] == team].tolist()
                    if team_row_index:
                        rank_cell = f'{pool_a_rank_col}{team_row_index[0] + pool_a_start_row}'
                        updates[rank_cell] = rank
                elif pool == 'B':
                    team_row_index = df.index[df[5] == team].tolist()
                    if team_row_index:
                        rank_cell = f'{pool_b_rank_col}{team_row_index[0] + pool_b_start_row}'
                        updates[rank_cell] = rank
                else:
                    print(f"No row found for team {team} in pool {pool}")

        if updates:
            self.scoreboard_manager.batch_write_scoreboard_updates(updates)
        else:
            print("No updates to apply to the scoreboard.")

In [ ]:
# Create an instance of the TieBreaker class
tie_breaker = TieBreaker(scoreboard_manager, rooms)

# Build the tournament DataFrame from the original scoreboard DataFrame
tournament_df = tie_breaker.build_tournament_dataframe(df)

# Calculate the final rankings using the tie-breaking process
tie_breaker.calculate_final_rankings(tournament_df)

# Retrieve the final rankings
final_rankings = tie_breaker.final_rankings

# Printing final rankings
tie_breaker.print_final_rankings()

# Update the scoreboard with the final rankings
tie_breaker.update_scoreboard_with_final_rankings(df)

In [ ]:
class Playoffs:
    def __init__(self, tie_breaker, rooms):
        self.tie_breaker = tie_breaker
        self.rooms = rooms  # Dictionary of Room objects
        self.final_standings = {}  # Store final standings after playoffs


    def setup_playoff_matches(self):
        # Use final rankings from tie_breaker to set up semi-finals and finals
        final_rankings = self.tie_breaker.final_rankings
        semi_finals = self.determine_semi_finals(final_rankings)

        # Set up semi-finals in Room 1 and Room 2
        self.setup_room_for_round('Room 1', semi_finals['semi_final_1'])
        self.setup_room_for_round('Room 2', semi_finals['semi_final_2'])

        return semi_finals


    def determine_semi_finals(self, final_rankings):
        # Logic to pair teams for semi-finals
        semi_finals = {}
        pool_a_winner = [team for team, rank in final_rankings['A'].items() if rank == 1][0]
        pool_b_winner = [team for team, rank in final_rankings['B'].items() if rank == 1][0]
        pool_a_runner_up = [team for team, rank in final_rankings['A'].items() if rank == 2][0]
        pool_b_runner_up = [team for team, rank in final_rankings['B'].items() if rank == 2][0]

        semi_finals['semi_final_1'] = (pool_a_winner, pool_b_runner_up)
        semi_finals['semi_final_2'] = (pool_b_winner, pool_a_runner_up)

        return semi_finals


    def setup_room_for_round(self, room_name, round):
        # Set up a room for a specific round
        room = self.rooms[room_name]
        room_updates = room.reset_room(1)  # Reset room for playoffs
        team_updates = room.update_teams(round)
        combined_updates = {**room_updates, **team_updates}
        worksheet = room.sheet.get_worksheet(0)
        room.batch_write_updates(worksheet, combined_updates)


    def setup_finals_and_consolation(self):
        # Determine and set up finals and consolation matches
        semi_final_results = self.final_standings
        final_match = (semi_final_results['Room 1']['Winner'], semi_final_results['Room 2']['Winner'])
        consolation_match = (semi_final_results['Room 1']['Loser'], semi_final_results['Room 2']['Loser'])

        # Set up finals in Room 1 and consolation in Room 2
        self.setup_room_for_round('Room 1', final_match)
        self.setup_room_for_round('Room 2', consolation_match)

In [ ]:
# playoffs code
playoffs = Playoffs(tie_breaker, rooms)
playoffs.setup_playoff_matches()

# Playoffs recording loop
print("Starting playoffs...")
while True:
    matches_completed = True

    for room_name in ['Room 1', 'Room 2']:
        room = rooms[room_name]
        worksheet = room.sheet.get_worksheet(0)
        room_df = room.read_entire_sheet(worksheet)

        print(f"Checking completion for {room_name}...")
        if room.check_round_completion(room_df):
            # Record match results
            scores = room.read_current_scores(room_df)
            room.store_room_data(room_name.split()[1], 1, room_df)  # Store round data
            print(f"Match in {room_name} is complete. Results: {scores}")

            # Logic for setting up finals/consolation based on semi-finals results goes here
        else:
            matches_completed = False
            print(f"{room_name} not yet complete.")

    if matches_completed:
        # logic to proceed to finals/consolation
        break

    print("Please wait before the next check...")
    time.sleep(30)
